# Tweet Sentiment Analysis

This project uses Machine Learning models to classify tweets into a labelled sentiment using the Twitter Sentiment Analysis dataset from Kaggle (https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis?resource=download) which focuses on Entity-level sentiment analysis on multi-lingual tweets.

### Importing libraries

First we import the libraries we will need in the project and check the dataset.

In [58]:
import pandas as pd
import numpy as np
import nltk
from nltk.collocations import *
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from collections import defaultdict
import re
import string
import itertools as it
import emoji
import fileinput
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import confusion_matrix,roc_auc_score,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import load_model

%matplotlib inline

The NLTK resources will be downloaded (which is why this cell should only be run *once*) and used to tokenize the words in each tweet.

In [32]:
#Download resources
#pre-trained sentence tokenizer
nltk.download('punkt')
#list of stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bcancinomeyer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bcancinomeyer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Exploratory Data Analysis

We begin by looking at our data so we know what we have to work with and what we will cleanup. We begin our EDA by opening both of our datasets as Pandas Dataframes.

In [33]:
df = pd.read_csv('twitter_training.csv', sep=',', names=["Tweet ID", "Entity", "Sentiment", "Tweet Content"])
df.head()
valid_df = pd.read_csv('twitter_validation.csv', sep=',', names=["Tweet ID", "Entity", "Sentiment", "Tweet Content"])

In [43]:
df["Tweet ID"] = range(1, len(df) + 1)
valid_df["Tweet ID"] = range(1, len(valid_df) + 1)
print(df.head(), valid_df.head())

   Tweet ID       Entity Sentiment  \
0         1  Borderlands  Positive   
1         2  Borderlands  Positive   
2         3  Borderlands  Positive   
3         4  Borderlands  Positive   
4         5  Borderlands  Positive   

                                       Tweet Content  
0  im getting on borderlands and i will murder yo...  
1  i am coming to the borders and i will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...      Tweet ID     Entity Sentiment  \
0         1   Facebook   Neutral   
1         2     Amazon   Neutral   
2         3  Microsoft  Negative   
3         4      CS-GO  Negative   
4         5     Google   Neutral   

                                       Tweet Content  
0  i mentioned on facebook that i was struggling ...  
1  bbc news - amazon boss jeff bezos rejects clai...  
2  @microsoft why do i pay for word when it funct...  
3  c

In [35]:
df['Tweet Content'][0]

'im getting on borderlands and i will murder you all ,'

### Data cleaning

We begin our cleaning by dropping any duplicate and NAN values present in both dataframes.

In [36]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
valid_df.dropna(inplace=True)
valid_df.drop_duplicates(inplace=True)

In [44]:
df['Sentiment'].replace(to_replace='Irrelevant', value='Neutral', inplace=True)
valid_df['Sentiment'].replace(to_replace='Irrelevant', value='Neutral', inplace=True)
print(set(df['Sentiment']))

{'Positive', 'Negative', 'Neutral'}


In [45]:
non_strings_mask = pd.to_numeric(df['Tweet Content'], errors='coerce').isna()
df.loc[non_strings_mask]
non_strings_mask = pd.to_numeric(valid_df['Tweet Content'], errors='coerce').isna()
valid_df.loc[non_strings_mask]

,Tweet ID,Entity,Sentiment,Tweet Content
0,1,Facebook,Neutral,i mentioned on facebook that i was struggling ...
1,2,Amazon,Neutral,bbc news - amazon boss jeff bezos rejects clai...
2,3,Microsoft,Negative,@microsoft why do i pay for word when it funct...
3,4,CS-GO,Negative,"csgo matchmaking is so full of closet hacking,..."
4,5,Google,Neutral,now the president is slapping americans in the...
...,...,...,...,...
995,996,GrandTheftAuto(GTA),Neutral,toronto is the arts and culture capital of ca...
996,997,CS-GO,Neutral,this is actually a good move tot bring more vi...
997,998,Borderlands,Positive,today sucked so its time to drink wine n play ...
998,999,Microsoft,Positive,bought a fraction of microsoft today. small wins.


In [47]:
#remove urls & special characters
def remove_urls(text):
    """Berilgan matndan URL larini o'chiradi"""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_emojis(text):
    """Berilgan matndan emojilarni o'chiradi"""
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emojilar
                               u"\U0001F300-\U0001F5FF"  # simvollar va diagrammalar
                               u"\U0001F680-\U0001F6FF"  # transport va turli joylar
                               u"\U0001F1E0-\U0001F1FF"  # davlat bayroqlari
                               u"\U00002702-\U000027B0"  # dingbats
                               u"\U000024C2-\U0001F251"  # alamatchilik belgilari
                               u"\U0001f300-\U0001f650"  
                               u"\u2000-\u3000" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df['Tweet Content'] = df['Tweet Content'].apply(lambda x: remove_emojis(x))
df['Tweet Content'] = df['Tweet Content'].apply(lambda x: remove_urls(x))
valid_df['Tweet Content'] = valid_df['Tweet Content'].apply(lambda x: remove_emojis(x))
valid_df['Tweet Content'] = valid_df['Tweet Content'].apply(lambda x: remove_urls(x))


df['Tweet Content'] = df['Tweet Content'].apply(str.lower)
valid_df['Tweet Content'] = valid_df['Tweet Content'].apply(str.lower)

stopwords_list = stopwords.words('english')
stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
stopwords_list += list(string.punctuation)
stopwords_list += ['....','...', '..', '’', "''", '``', '-', "'", "([a-zA-Z]+(?:'[a-z]+)?)"]

print(df.tail())
print(valid_df.tail())

       Tweet ID  Entity Sentiment  \
74677     73992  Nvidia  Positive   
74678     73993  Nvidia  Positive   
74679     73994  Nvidia  Positive   
74680     73995  Nvidia  Positive   
74681     73996  Nvidia  Positive   

                                           Tweet Content  
74677  just realized that the windows partition of my...  
74678  just realized that my mac window partition is ...  
74679  just realized the windows partition of my mac ...  
74680  just realized between the windows partition of...  
74681  just like the windows partition of my mac is l...  
     Tweet ID               Entity Sentiment  \
995       996  GrandTheftAuto(GTA)   Neutral   
996       997                CS-GO   Neutral   
997       998          Borderlands  Positive   
998       999            Microsoft  Positive   
999      1000      johnson&johnson   Neutral   

                                         Tweet Content  
995   toronto is the arts and culture capital of ca...  
996  this is actuall

In [46]:
word_tokenize(df['Tweet Content'][74677])

['just',
 'realized',
 'that',
 'the',
 'windows',
 'partition',
 'of',
 'my',
 'mac',
 'is',
 'like',
 '6',
 'years',
 'behind',
 'nvidia',
 'drivers',
 'and',
 'i',
 'have',
 'no',
 'idea',
 'how',
 'i',
 'did',
 'not',
 'notice']

In [6]:
#Defining entity dictionary
entity_dict = { 
                "RedDeadRedemption(RDR)" : ['rdr', 'red dead redemption', 'red dead'], 
                "Microsoft": ['microsoft'],
                "Xbox(XSeries)":['xbox', 'series x', 'series s', 'xbox one', 'xseries'], 
                "AssassinsCreed": ['assassinscreed', 'assassins creed'], 
                "CallOfDutyBlackopsColdWar": ['black ops', 'cold war', 'callOfdutyblackopscoldWar'],
                "FIFA": ['fifa'],
                "TomClancysGhostRecon": ['ghost recon', 'ghostrecon'],
                "Google": ['google'],
                "PlayStation(PS)": ['ps5', 'playstation', 'ps4', 'PS'],
                "Facebook": ['facebook'],
                "GrandTheftAuto(GTA)": ['gta', 'grand theft auto'],
                "PlayerUnknownsBattlegrounds(PUBG)": ['pubg', 'player unknowns battlegrounds', 'PlayerUnknownsBattlegrounds'],
                "Hearthstone": ['hearthstone'],
                "MaddenNFL": ['madden'],
                "CallOfDuty": ['modern warfare', 'call of duty', 'cod'],
                "Fortnite": ['fortnitegame', 'fortnite'],
                "Verizon": ['verizon'],
                "Nvidia": ['nvidia'],
                "Amazon": ['amazon'],
                "WorldOfCraft": ['wow', 'world of warcraft'],
                "ApexLegends": ['apex legends', 'apex', 'apexlegends'],
                "CS-GO": ['csgo', 'counter strike'],
                "johnson&johnson": ['johnson&johnson', 'johnson & johnson'],
                "HomeDepot": ['homedepot', 'home depot'],
                "NBA2K": ['nba'],
                "Overwatch": ['overwatch'],
                "LeagueOfLegends": ['lol', 'league of legends'],
                "Borderlands": ['borderlands'],
                "TomClancysRainbowSix": ['rainbow six', 'rainbow six siege', 'rainbowsix'],
                "Dota": ['dota'],
                "Battlefield": ['battlefield'],
                "Cyberpunk2077": ['cyberpunkgame', 'cyberpunk2077', 'cyberpunk'],
                "NintendoSwitch": ['nintendo switch', 'nintendo'],
                "Windows": ['windows', 'window']
              }

In [12]:
def tokenize_text(text):
    return word_tokenize(text)

df['Tokenized Text'] = df['Tweet Content'].apply(tokenize_text)


df['Tokenized Text'] = [word for word in df['Tokenized Text'] if word not in stopwords_list]
df['Tokenized Text'] = [word for word in df['Tokenized Text'] if word not in stopwords_list]
df.head()

,Tweet ID,Entity,Sentiment,Tweet Content,Tokenized Text
0,1,Borderlands,Positive,im getting on borderlands and i will murder yo...,"[im, getting, on, borderlands, and, i, will, m..."
1,2,Borderlands,Positive,i am coming to the borders and i will kill you...,"[i, am, coming, to, the, borders, and, i, will..."
2,3,Borderlands,Positive,im getting on borderlands and i will kill you ...,"[im, getting, on, borderlands, and, i, will, k..."
3,4,Borderlands,Positive,im coming on borderlands and i will murder you...,"[im, coming, on, borderlands, and, i, will, mu..."
4,5,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,"[im, getting, on, borderlands, 2, and, i, will..."


In [13]:
df['Tokenized Text'][0]

['im',
 'getting',
 'on',
 'borderlands',
 'and',
 'i',
 'will',
 'murder',
 'you',
 'all',
 ',']

## Creating a Long Short-Term Memory (LTSM) Model

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Tokenized Text'])
sequences = tokenizer.texts_to_sequences(df['Tokenized Text'])
X = pad_sequences(sequences)

In [15]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Sentiment'])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
vocab_size = len(tokenizer.word_index) + 1
max_sequence_length = X.shape[1]
embedding_dim = 100
lstm_units = 128

In [18]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
y_train_encoded = np.eye(len(label_encoder.classes_))[y_train]
y_test_encoded = np.eye(len(label_encoder.classes_))[y_test]

In [23]:
batch_size = 64
epochs = 10
model.fit(X_train, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.2)


Epoch 1/10
740/740 [==============================] - 464s 624ms/step - loss: 0.7342 - accuracy: 0.6774 - val_loss: 0.5658 - val_accuracy: 0.7676
Epoch 2/10
740/740 [==============================] - 481s 650ms/step - loss: 0.3581 - accuracy: 0.8624 - val_loss: 0.3990 - val_accuracy: 0.8456
Epoch 3/10
740/740 [==============================] - 508s 687ms/step - loss: 0.2219 - accuracy: 0.9139 - val_loss: 0.3638 - val_accuracy: 0.8651
Epoch 4/10
740/740 [==============================] - 492s 665ms/step - loss: 0.1612 - accuracy: 0.9363 - val_loss: 0.3483 - val_accuracy: 0.8742
Epoch 5/10
740/740 [==============================] - 538s 727ms/step - loss: 0.1386 - accuracy: 0.9451 - val_loss: 0.3925 - val_accuracy: 0.8721
Epoch 6/10
740/740 [==============================] - 489s 660ms/step - loss: 0.1137 - accuracy: 0.9545 - val_loss: 0.3735 - val_accuracy: 0.8801
Epoch 7/10
740/740 [==============================] - 488s 659ms/step - loss: 0.1006 - accuracy: 0.9590 - val_loss: 0.3819 -

In [24]:
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

463/463 [==============================] - 20s 42ms/step - loss: 0.4390 - accuracy: 0.8862
Test Loss: 0.4390312433242798, Test Accuracy: 0.8862162232398987


In [27]:
model.save('tweetsentiment_lstm_model.h5')

In [51]:
loaded_model = load_model("tweetsentiment_lstm_model.h5")

In [66]:
new_sequences = tokenizer.texts_to_sequences(valid_df["Tweet Content"])
new_X = pad_sequences(new_sequences, maxlen=max_sequence_length)

In [67]:
predictions = loaded_model.predict(new_X)

32/32 [==============================] - 1s 36ms/step


In [71]:
predicted_prob = 
class_predictions = np.argmax(class_predictions, axis=1)

actual_labels = valid_df["Sentiment"]

In [70]:
cm = confusion_matrix(actual_labels, class_predictions)

TypeError: Singleton array 0 cannot be considered a valid collection.